## Envrionment Setup

- Run `conda env create --name avalon --file=environment.yaml`
- Then switch to the environment by clicking the `avalon` item of the drop-down in the top right corner of Jupyter Notebook.

# Crime Forecast in Vancouver

by Ben Chen, Mo Norouzi, Orix Au Yeung, Yiwei Zhang

In [1]:
import altair as alt
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from pandas.plotting import autocorrelation_plot
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_absolute_error, mean_squared_error
import warnings
import sys
from myst_nb import glue
warnings.filterwarnings("ignore")
sys.path.append('../src') 

from get_metrics_from_df import get_metrics_from_df
from missing_value import missing_zero_values_table
from numeric_cols import create_numeric_cols_chart
from merge_forecast_values import merge_forecast_values
from arima_forecasting import arima_prediction
from correlation import get_redundant_pairs, get_top_abs_correlations

In [2]:
data = pd.read_csv("../data/raw/crimedata_csv_AllNeighbourhoods_AllYears.csv", encoding="utf-8")
glue("data_head", data.head().style.format().hide(), display=False)

data_description = pd.read_csv("../results/tables/description.csv", encoding="utf-8")
glue("data_description", data_description.head().style.format().hide(), display=False)

# Summary

In this notebook, our focus revolved around constructing a time-series forecasting model tailored to predict crime incidents in Vancouver, using "Month" as the temporal unit. Our primary emphasis centered on one of the most prevalent crime types in Vancouver over the past two decades: theft from vehicles. We evaluated the efficacy of three fundamental forecasting models—simple moving average, exponential smoothing, and ARIMA(1,1,0). Notably, the ARIMA(1,1,0) model emerged as the most effective, yielding a Mean Absolute Error (MAE) of 26.856890. Considering that the occurrences of "theft from a vehicle" crimes per month often range in the hundreds to thousands, achieving a forecast performance of this caliber is notably commendable. It's worth highlighting that further refinement through comprehensive parameter tuning and integration of additional external variables holds the potential to cultivate even more accurate forecasting models.

# Introduction

Vehicle-related theft remains an ongoing concern nationwide in Canada, with statistics revealing a staggering incident of vehicle theft occurring every six minutes across the country {cite}`CTVnews`. This pervasive issue extends into Vancouver, presenting formidable challenges to both community safety and law enforcement efforts. Theft from vehicles, a prevalent form of this crime, significantly affects neighborhoods, inflicting distress and substantial financial losses on local residents. In response to this pressing concern, this project is dedicated to forecasting occurrences of theft from vehicles specifically within Vancouver.

The primary objective of this project is to forecast instances of theft from vehicles in Vancouver by analyzing historical data. Leveraging a comprehensive dataset sourced from the Vancouver Police Department, encompassing diverse crime records in Vancouver over the past 20 years alongside incident locations, our goal is to construct a reliable predictive model. This model aims to anticipate the frequency and patterns specific to theft from vehicles. An accurate forecast holds the potential to empower the City of Vancouver to proactively allocate law enforcement resources, thereby curbing the occurrence of such crimes and enhancing community safety.

# Methods

## Data

The dataset utilized for this project originates from the Vancouver Police Department, available through the following link: https://geodash.vpd.ca/opendata/ {cite}`crimedata2023`. It comprises 10 columns/variables and encompasses a substantial volume of data, totaling 879,861 rows. Each row corresponds to a distinct crime incident recorded within the dataset. The available information includes details about the crime type, the corresponding date of occurrence, and the specific location or neighborhood where the crime took place. These data points serve as crucial elements for our analysis and forecasting efforts.

## Analysis

We're deploying three distinct time-series forecasting models—Simple Moving Average (SMA), Exponential Smoothing (ES), and Autoregressive Integrated Moving Average (ARIMA). These models rely solely on the timestamp and the targeted forecasted value. Despite having location data, which holds potential value, we've deferred its utilization in this phase of the project. Employing a rolling window approach, we'll predict and assess model performance across a 20-year duration, setting the window size to 12 months. This configuration ensures that forecasts leverage the preceding year's data for accuracy. Specifically for ARIMA, the hyperparameters (p, d, q) are set at (1, 1, 0). This specification signifies that the model factors in the most recent lagged observations of the differenced series to predict the subsequent value. Our analysis was executed using Python, leveraging various libraries: numpy {cite}`harris2020`, Pandas {cite}`mckinney2010`, Altair {cite}`vanderplas2018`, scikit-learn {cite}`pedregosa`, Matplotlib {cite}`hunter2007`, Seaborn {cite}`seaborn`, and Statsmodels {cite}`seabold2010`.

# Results & Discussions

Upon conducting exploratory data analysis (EDA), conspicuous anomalies surface in the dataset. The HOUR and MINUTE columns exhibit an unusual frequency of zero values, along with a disproportionate occurrence of '30' in the MINUTE column. Additionally, the DAY column prominently features an excessive number of records logged on the 31st of the month. These irregularities likely stem from convenience in data recording, casting uncertainty on the accuracy of these three columns. In light of these inconsistencies, the most prudent approach is to exclude the DAY, HOUR, and MINUTE columns from analysis and focus solely on forecasting crime occurrences based on the MONTH variable.

```{glue:figure} data_head
:name: "data_head"

Data Preview
```

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 879861 entries, 0 to 879860
Data columns (total 10 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   TYPE           879861 non-null  object 
 1   YEAR           879861 non-null  int64  
 2   MONTH          879861 non-null  int64  
 3   DAY            879861 non-null  int64  
 4   HOUR           879861 non-null  int64  
 5   MINUTE         879861 non-null  int64  
 6   HUNDRED_BLOCK  879849 non-null  object 
 7   NEIGHBOURHOOD  879717 non-null  object 
 8   X              879785 non-null  float64
 9   Y              879785 non-null  float64
dtypes: float64(2), int64(5), object(3)
memory usage: 67.1+ MB


```{glue:figure} data_description
:name: "data_description"

Data Summary Statistics
```

In [4]:
data.describe().T

,count,mean,std,min,25%,50%,75%,max
YEAR,879861.0,2.012265e+03,6.183902e+00,2003.0,2.006000e+03,2.012000e+03,2.018000e+03,2023.0
MONTH,879861.0,6.516683e+00,3.391857e+00,1.0,4.000000e+00,7.000000e+00,9.000000e+00,12.0
DAY,879861.0,1.538500e+01,8.757135e+00,1.0,8.000000e+00,1.500000e+01,2.300000e+01,31.0
HOUR,879861.0,1.231342e+01,7.463913e+00,0.0,7.000000e+00,1.400000e+01,1.800000e+01,23.0
MINUTE,879861.0,1.586139e+01,1.836042e+01,0.0,0.000000e+00,5.000000e+00,3.000000e+01,59.0
X,879785.0,4.490074e+05,1.393043e+05,0.0,4.901879e+05,4.915699e+05,4.933801e+05,511303.0
Y,879785.0,4.977853e+06,1.544127e+06,0.0,5.454211e+06,5.457170e+06,5.458718e+06,5512579.0


### Missing values

In [5]:
missing_zero_values_table(data)

Your selected dataframe has 10 columns and 879861 Rows.
There are 4 columns that have missing values.


,Zero Values,Missing Values,% of Total Values,Total Zero Missing Values,% Total Zero Missing Values,Data Type
NEIGHBOURHOOD,0,144,0.0,144,0.0,object
X,77225,76,0.0,77301,8.8,float64
Y,77225,76,0.0,77301,8.8,float64
HUNDRED_BLOCK,0,12,0.0,12,0.0,object


### Distribution

```{figure} ../results/figures/numeric_dist.png
---
width: 800px
name: numeric_dist
---
The distribution of numerical variables.
```

```{figure} ../results/figures/categ_dist.png
---
width: 600px
name: categ_dist
---
The distribution of categorical variables.
```

### Correlation

In [6]:
print("Top Absolute Correlations !")
print(get_top_abs_correlations(data.select_dtypes(include=['int32','int64']), 10))

Top Absolute Correlations !
HOUR   MINUTE    0.114717
YEAR   MINUTE    0.056099
       HOUR      0.035971
       MONTH     0.010681
       DAY       0.009736
MONTH  DAY       0.006062
DAY    HOUR      0.004696
MONTH  MINUTE    0.003963
DAY    MINUTE    0.003185
MONTH  HOUR      0.002013
dtype: float64


### Preprocessing

We'll start the data preprocessing phase by grouping the rows according to the TYPE, YEAR, and MONTH columns to aggregate the counts of specific crimes occurring in each month. Additionally, we'll adjust the datetime variable format for consistency. However, as the latest month (2023-11) is incomplete, we'll exclude this month from the dataset. Finally, we'll filter the data so that we focus only on `Theft from Vehicle` crimes, the most common crime in Vancouver in the past 20 years. This initial processing sets the groundwork for our subsequent time-series forecasting models.

In [7]:
preprocessed_data_full = pd.read_csv("../data/processed/preprocessed_data_full.csv", encoding="utf-8")
preprocessed_data_full.head()

,TYPE,YEAR,MONTH,Observations,YEAR-MONTH
0,Break and Enter Commercial,2003,1,303,2003-01-01
1,Break and Enter Commercial,2003,2,254,2003-02-01
2,Break and Enter Commercial,2003,3,292,2003-03-01
3,Break and Enter Commercial,2003,4,266,2003-04-01
4,Break and Enter Commercial,2003,5,290,2003-05-01


In [8]:
theft_from_vehicle = pd.read_csv("../data/processed/preprocessed_theft_from_vehicle_full.csv", encoding="utf-8")
theft_from_vehicle.head()

,TYPE,YEAR,MONTH,Observations,YEAR-MONTH
0,Theft from Vehicle,2003,1,1438,2003-01-01
1,Theft from Vehicle,2003,2,1102,2003-02-01
2,Theft from Vehicle,2003,3,1251,2003-03-01
3,Theft from Vehicle,2003,4,1528,2003-04-01
4,Theft from Vehicle,2003,5,1873,2003-05-01


In [9]:
theft_from_vehicle.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   TYPE          250 non-null    object
 1   YEAR          250 non-null    int64 
 2   MONTH         250 non-null    int64 
 3   Observations  250 non-null    int64 
 4   YEAR-MONTH    250 non-null    object
dtypes: int64(3), object(2)
memory usage: 9.9+ KB


```{figure} ../results/figures/original_plot.png
---
width: 600px
name: original_plot
---
The time-series trend of "Theft from Vehicle" in Vancouver.
```

In [10]:
theft_from_vehicle_filtered = theft_from_vehicle[['YEAR-MONTH','Observations']]
theft_from_vehicle_filtered.set_index('YEAR-MONTH', inplace=True)
theft_from_vehicle_filtered.head()

,Observations
YEAR-MONTH,
2003-01-01,1438
2003-02-01,1102
2003-03-01,1251
2003-04-01,1528
2003-05-01,1873


### Simple Moving Average & Exponential Smoothing

In [11]:
prediction = pd.read_csv("../results/tables/all_predictions.csv", index_col=[0])
prediction.index = pd.to_datetime(prediction.index)
prediction.head(15)

,Observations,SMA_Forecast,ES_Forecast,ARIMA_Forecast
2003-01-01,1438.0,NaN,NaN,NaN
2003-02-01,1102.0,NaN,NaN,NaN
2003-03-01,1251.0,NaN,NaN,NaN
2003-04-01,1528.0,NaN,NaN,NaN
2003-05-01,1873.0,NaN,NaN,NaN
2003-06-01,1695.0,NaN,NaN,NaN
2003-07-01,1445.0,NaN,NaN,NaN
2003-08-01,1634.0,NaN,NaN,NaN
2003-09-01,1470.0,NaN,NaN,NaN
2003-10-01,1554.0,NaN,NaN,NaN


In [12]:
prediction.tail(5)

,Observations,SMA_Forecast,ES_Forecast,ARIMA_Forecast
2023-07-01,684.0,594.083333,603.749242,665.145976
2023-08-01,749.0,603.250000,628.475010,717.025302
2023-09-01,651.0,613.916667,665.449143,701.821954
2023-10-01,656.0,611.500000,660.768368,652.819444
2023-11-01,NaN,611.583333,657.150934,652.854250


```{figure} ../results/figures/sma_prediction_plot.png
---
width: 600px
name: sma_prediction_plot
---
The plot that compares the original value and the predictions made by simple moving average.
```

```{figure} ../results/figures/es_prediction_plot.png
---
width: 600px
name: es_prediction_plot
---
The plot that compares the original value and the predictions made by exponential smoothing.

Based on a visual assessment of the Simple Moving Average (SMA) and Exponential Smoothing (ES) forecasts, it's evident that both methods broadly capture the general trend of the actual values. However, neither forecast method appears to be highly accurate. The Exponential Smoothing approach demonstrates a slightly improved performance compared to SMA.

### ARIMA(1,1,0)

```{figure} ../results/figures/autocorrelation_plot.png
---
width: 600px
name: autocorrelation_plot
---
The autocorrelation plot for the time-series observations in our preprocessed data.

```{figure} ../results/figures/autocorrelation_with_diff_plot.png
---
width: 600px
name: autocorrelation_with_diff_plot
---
The autocorrelation plot for the time-series observations in our preprocessed data with 1-time differencing.

```{figure} ../results/figures/arima_prediction_plot.png
---
width: 600px
name: arima_prediction_plot
---
The plot that compares the original value and the predictions made by arima model.
```

The forecast from the ARIMA model looks much better! We can see some clear overlaps between the forecasted value and the original value.

In [13]:
# Drop NA values to evaluate performance

prediction_drop_na = prediction.dropna()

results_df = get_metrics_from_df(prediction_drop_na, "Observations")
results_df

,Forecast_Column,MAE,MSE
0,SMA_Forecast,121.216737,27239.951827
1,ES_Forecast,98.641493,16240.436389
2,ARIMA_Forecast,26.856890,1940.870649


The displayed dataframe outlines the performance metrics, specifically the mean absolute error (MAE) and mean squared error (MSE), for the three models. Notably, there's a discernible pattern showcasing a marked enhancement in performance, progressing from Simple Moving Average (SMA) to Exponential Smoothing Approach (ESA) and ultimately to ARIMA. This consistent trend aligns with the observations gleaned from the visualizations crafted earlier, affirming the gradual improvement in forecasting accuracy across the models.

While the ARIMA model stands out as the most effective among the three forecasting models—simple moving average and exponential smoothing—it's crucial to acknowledge the room for enhancement in our predictive capabilities. Future advancements could entail fine-tuning the ARIMA hyperparameters or exploring alternative models to ascertain if further accuracy gains are attainable. Additionally, integrating exogenous variables, such as socioeconomic indicators or weather data, might augment the predictive power of our models, offering a more comprehensive understanding of crime dynamics. Furthermore, this analysis prompts future inquiries, including investigating the influence of specific external factors on crime occurrences or exploring spatial-temporal models to predict crime hotspots within Vancouver. These prospective avenues aim to refine our forecasting precision and deepen our insights into crime trends, paving the way for more informed law enforcement strategies and proactive crime prevention measures.

# References

```{bibliography}
```